# Tutorial Mengolah Data dengan Pandas

Selamat datang di tutorial dasar pengolahan data menggunakan Pandas di Python. Pandas adalah library yang sangat powerful untuk manipulasi dan analisis data.

Dalam tutorial ini, kita akan belajar:
1. Mengimpor Pandas
2. Memuat data dari file CSV
3. Mengeksplorasi data
4. Membersihkan data
5. Memanipulasi data
6. Grouping dan agregasi
7. Menggabungkan DataFrames
8. Mengekspor data

Mari kita mulai!

## 1. Mengimpor Pandas

Pertama, kita perlu mengimpor library Pandas. Jika belum terinstall, install dengan `pip install pandas`.

In [8]:
import pandas as pd

print("Pandas version:", pd.__version__)

Pandas version: 2.3.3


## 2. Memuat Data

Pandas dapat membaca data dari berbagai sumber seperti CSV, Excel, JSON, SQL database, dll. Kita akan menggunakan file CSV dari workspace kita: `shipments/new-shipments-parsed.csv`.

In [9]:
# Membaca file CSV (beberapa baris memiliki format yang tidak konsisten, jadi kita skip bad lines)
df = pd.read_csv('shipments/new-shipments-parsed.csv', engine='python', on_bad_lines='skip')

# Menampilkan beberapa baris pertama
df.head()

,customer_name,box_size,grade,top_color,pallets,cases,notes
0,General Meats,11lb,Large,White Top,3.0,NaN,NaN
1,General Meats,4lb,Large,White Top,3.0,NaN,NaN
2,General Meats,2lb,Large,White Top,1.0,NaN,NaN
3,General Meats,9lb,Large,NaN,1.0,NaN,in case it's available
4,International Market (SHAY),11lb,Choice,Red Orchid Top,4.0,NaN,NaN


## 3. Mengeksplorasi Data

Setelah memuat data, kita perlu memahami strukturnya. Gunakan method seperti `head()`, `info()`, `describe()`, dan `shape()`.

In [10]:
# Informasi dasar tentang DataFrame
print("Shape:", df.shape)
print("\nInfo:")
df.info()

print("\nDescribe:")
df.describe()

# Menampilkan kolom
print("\nColumns:", df.columns.tolist())

Shape: (55, 7)

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   customer_name  55 non-null     object 
 1   box_size       55 non-null     object 
 2   grade          55 non-null     object 
 3   top_color      26 non-null     object 
 4   pallets        50 non-null     float64
 5   cases          11 non-null     float64
 6   notes          11 non-null     object 
dtypes: float64(2), object(5)
memory usage: 3.1+ KB

Describe:

Columns: ['customer_name', 'box_size', 'grade', 'top_color', 'pallets', 'cases', 'notes']


## 4. Membersihkan Data

Data seringkali memiliki nilai yang hilang, duplikat, atau tipe data yang salah. Kita akan belajar menangani ini.

### Mengapa Menangani Data Hilang Penting?

Data hilang (missing values) dapat mempengaruhi analisis dan model machine learning. Kita tidak selalu "menghapus" data hilang, tapi menanganinya dengan cara:

- **Mengisi (impute)**: Mengganti nilai hilang dengan nilai rata-rata, median, modus, atau nilai tetap (seperti 0 atau string kosong).
- **Menghapus**: Menghapus baris atau kolom yang memiliki nilai hilang, jika data hilang terlalu banyak atau tidak penting.
- **Mengabaikan**: Untuk analisis tertentu, algoritma dapat menangani missing values secara otomatis.

Pilihan tergantung pada konteks: jika data hilang sedikit, isi saja; jika banyak, pertimbangkan menghapus. Menghapus data hilang dapat mengurangi ukuran dataset, tapi mengisi dapat memperkenalkan bias.

### Mengapa Menghapus Data Duplikat?

Data duplikat terjadi ketika baris yang sama muncul lebih dari sekali. Ini dapat menyebabkan masalah seperti:

- **Bias dalam analisis**: Statistik seperti rata-rata atau jumlah menjadi tidak akurat karena data yang sama dihitung berkali-kali.
- **Penyimpanan yang boros**: Membuang ruang penyimpanan dan memperlambat pemrosesan.
- **Kesalahan interpretasi**: Misalnya, jika menghitung jumlah pelanggan unik, duplikat akan membuat angka terlalu tinggi.

Kita menggunakan `drop_duplicates()` untuk menghapus baris yang identik. Jika ada kolom tertentu yang tidak boleh duplikat (seperti ID), gunakan parameter `subset`.

In [11]:
# Mengecek nilai yang hilang
print("Missing values:")
print(df.isnull().sum())

# Mengisi nilai yang hilang atau menghapusnya
df_clean = df.fillna('')  # Mengisi dengan string kosong untuk contoh

# Menghapus duplikat jika ada
df_clean = df_clean.drop_duplicates()

print("\nData setelah cleaning:")
df_clean.head()

Missing values:
customer_name     0
box_size          0
grade             0
top_color        29
pallets           5
cases            44
notes            44
dtype: int64

Data setelah cleaning:


,customer_name,box_size,grade,top_color,pallets,cases,notes
0,General Meats,11lb,Large,White Top,3.0,,
1,General Meats,4lb,Large,White Top,3.0,,
2,General Meats,2lb,Large,White Top,1.0,,
3,General Meats,9lb,Large,,1.0,,in case it's available
4,International Market (SHAY),11lb,Choice,Red Orchid Top,4.0,,


## 5. Memanipulasi Data

Manipulasi data meliputi filtering, sorting, menambah kolom baru, dll.

In [ ]:
# Filtering data
large_boxes = df_clean[df_clean['grade'] == 'Large']
print("Boxes with Large grade:")
print(large_boxes.head())

# Sorting data
sorted_df = df_clean.sort_values(by='pallets', ascending=False)
print("\nSorted by pallets (descending):")
print(sorted_df.head())

# Menambah kolom baru
df_clean['total_items'] = df_clean['pallets'].fillna(0) + df_clean['cases'].fillna(0)
print("\nWith new column 'total_items':")
print(df_clean[['customer_name', 'pallets', 'cases', 'total_items']].head())

## 6. Grouping dan Agregasi

Gunakan `groupby()` untuk mengelompokkan data berdasarkan kolom tertentu dan menerapkan fungsi agregasi.

In [12]:
# Grouping berdasarkan customer_name dan menjumlahkan pallets
grouped = df_clean.groupby('customer_name')['pallets'].sum().reset_index()
print("Total pallets per customer:")
print(grouped)

# Agregasi multiple
agg_df = df_clean.groupby('grade').agg({
    'pallets': 'sum',
    'cases': 'sum',
    'customer_name': 'count'
}).reset_index()
print("\nAggregated by grade:")
print(agg_df)

Total pallets per customer:
                    customer_name pallets
0                        Blue Fig     8.5
1                    Cedar Market        
2   Daar Al-Islam Mosque St.Louis     3.0
3                           Deshi    20.0
4                    Famous Foods    20.0
5                   General Meats     8.0
6               Green Vine Market     2.0
7                        HolyLand     9.0
8                   India Farmers     8.0
9     International Market (SHAY)     8.0
10                    Kemal Pasha     6.0
11                   Lipari Foods    57.0
12                      New Royal    11.0
13                     OM produce    43.0


TypeError: unsupported operand type(s) for +: 'float' and 'str'

## 7. Menggabungkan DataFrames

Kita dapat menggabungkan beberapa DataFrames menggunakan `merge()`, `join()`, atau `concat()`. Mari kita baca file lain dan gabungkan.

In [ ]:
# Membaca file lain
orders_df = pd.read_csv('shipments/orders.csv')

# Merge berdasarkan customer_name
merged_df = pd.merge(df_clean, orders_df, on='customer_name', how='left', suffixes=('_shipments', '_orders'))

print("Merged DataFrame:")
print(merged_df.head())

## 8. Mengekspor Data

Setelah memproses data, kita dapat menyimpannya ke file baru.

In [ ]:
# Mengekspor ke CSV
df_clean.to_csv('processed_shipments.csv', index=False)
print("Data diekspor ke 'processed_shipments.csv'")

# Mengekspor ke Excel (jika ada openpyxl)
# df_clean.to_excel('processed_shipments.xlsx', index=False)

## Kesimpulan

Selamat! Anda telah belajar dasar-dasar pengolahan data dengan Pandas. Pandas sangat powerful untuk:

- Membaca dan menulis data dari berbagai format
- Membersihkan dan mempersiapkan data
- Melakukan analisis dan transformasi
- Menggabungkan data dari sumber berbeda

Lanjutkan eksplorasi dengan dataset Anda sendiri dan pelajari lebih lanjut di dokumentasi Pandas: https://pandas.pydata.org/docs/

Praktikkan dengan data di workspace ini!